### Load BioCaser Datasets

In [1]:
from utils.data_util import biocaser2text

data_file = "/home/zm324/workspace/doc_cls/data/biocaster/BioCaster.3.xml"
data_df = biocaser2text(data_file)
map_dic = {"negative": 0, "positive": 1}
data_df.labels = data_df.labels.apply(lambda x: map_dic[x])
data_df.head()

parse biocaser data from /home/zm324/workspace/doc_cls/data/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0
1,\nTyphoid outbreak in Agusan del Sur town unde...,1
2,\n Typhoid Outbreak In Central Nepal November...,1
3,\nChlorine truck crash shuts Turnpike exit\nPo...,0
4,\nMan dies from rare anthrax bug \nChristopher...,1


In [2]:
from utils.preprocess import PreProcess
pre_processor = PreProcess(data_df, "docs",lower=False)
# todo: change code to provide all functions in class definition.
pre_processor.clean_html()
pre_processor.remove_non_ascii()
pre_processor.remove_spaces()
pre_processor.remove_punctuation()
data_df.head()

,docs,labels
0,Bird Flu Outbreak Drill Spooks Manitoba Town ...,0
1,Typhoid outbreak in Agusan del Sur town under...,1
2,Typhoid Outbreak In Central Nepal November 7...,1
3,Chlorine truck crash shuts Turnpike exit Post...,0
4,Man dies from rare anthrax bug Christopher No...,1


In [3]:
word_to_ix = {}
for idx,row in data_df.iterrows():
    doc = row.docs
    for word in doc.split():
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

In [11]:
vocab_size = len(word_to_ix)
embed_dim = 300

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
initrange = 0.005
embedding.weight.data.uniform_(-initrange, initrange)

tensor([[-0.0017, -0.0041,  0.0032,  ..., -0.0032, -0.0004, -0.0030],
        [ 0.0010,  0.0012, -0.0050,  ..., -0.0026,  0.0019,  0.0035],
        [-0.0041,  0.0017, -0.0033,  ...,  0.0012,  0.0005,  0.0018],
        ...,
        [-0.0048, -0.0033, -0.0004,  ..., -0.0007, -0.0019,  0.0049],
        [ 0.0026, -0.0026,  0.0004,  ..., -0.0029, -0.0024, -0.0036],
        [-0.0043,  0.0048,  0.0014,  ...,  0.0026,  0.0045,  0.0014]])

In [13]:
text = torch.tensor([0,0,1])
offsets = torch.tensor([0])

In [ ]:
import sys
sys.path.append('../../')
import pandas as pd
from doc_cls.preprocess import PreProcess
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
import html
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm
import numpy as np

In [ ]:
from src.data_util import biocaser2text
data_file = "../../data/biocaster/BioCaster.3.xml"
data_df = biocaser2text(data_file)

In [ ]:
data_df.head()

In [ ]:
data_df["Category"].value_counts()

### Preprocessing

In [ ]:
column_name = data_df.columns[0]
    # print(column_name)
pre_processor = PreProcess(data_df, column_name)
# todo: change code to provide all functions in class definition.
data = pre_processor.clean_html()
data = pre_processor.remove_non_ascii()
data = pre_processor.remove_spaces()
data = pre_processor.remove_punctuation()
data = pre_processor.stemming()
data = pre_processor.lemmatization()
data = pre_processor.stop_words()

### Feature extraction

#### TFIDF

### Bag of Word

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
data["bow"] = list(vectorizer.fit_transform(data.Document.to_numpy()).toarray())
len(vectorizer.get_feature_names())

### Lowercase, Remove Stopwords, Stemming and lemmatization

In [ ]:
import torch
import torchtext
from torchtext.datasets import text_classification

NGRAMS = 2
import os
if not os.path.isdir('../data'):
    os.mkdir('../data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='../data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torchtext.data.utils import get_tokenizer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
tokenizer = get_tokenizer("basic_english")

docs = []
doc_string = doc_string.lower() # use only lowercase
#         doc_string = [w for w in doc_string if (w not in stopwords)]
#         doc_string = "".join(doc_string)
tokens = tokenizer(doc_string)
tokens = [w for w in tokens if (w not in stopwords) and (w.isalnum())]
break
#     docs.append(tokens)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [ ]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [ ]:
from torch.utils.data import DataLoader

def fit(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [ ]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 50
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

In [ ]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

In [ ]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])